# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ribeira grande,38.5167,-28.7000,18.22,77,20,7.72,PT,1706980777
1,1,broome,42.2506,-75.8330,1.41,78,40,4.12,US,1706980778
2,2,stephenville,32.2207,-98.2023,15.69,58,0,3.13,US,1706980568
3,3,minas de marcona,-15.2119,-75.1103,23.39,62,70,4.40,PE,1706980778
4,4,cabo san lucas,22.8909,-109.9124,23.12,55,0,8.75,MX,1706980669


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
# %%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points(
    x = 'Lng',
    y = 'Lat',
    size = 'Humidity',
    xlabel = 'Longitude',
    ylabel = 'Latitude',
    hover_cols = ['City'],
    geo=True,
    tiles='OSM',
    color = 'City',
    scale = 0.5,
    frame_width = 700,
    frame_height = 500)

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[(city_data_df['Max Temp']>=20) & (city_data_df['Max Temp']<=32) & (city_data_df['Humidity']<=60)]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,cabo san lucas,22.8909,-109.9124,23.12,55,0,8.75,MX,1706980669
5,5,maracas,-13.4411,-40.4308,28.74,34,89,3.02,BR,1706980778
9,9,fada,9.5667,8.3000,30.88,14,2,0.14,NG,1706980779
18,18,mbaiki,3.8678,17.9892,29.01,27,93,0.68,CF,1706980780
31,31,santa elena de uairen,4.6167,-61.1333,31.96,32,18,3.81,VE,1706980782
...,...,...,...,...,...,...,...,...,...,...
507,507,tanout,14.9709,8.8879,22.26,11,35,6.48,NE,1706980899
525,525,mul,20.0667,79.6667,22.58,32,0,1.77,IN,1706980902
546,546,lamarque,-39.4230,-65.7021,28.59,15,8,5.80,AR,1706980906
570,570,guerrero,17.6667,-100.0000,25.23,37,1,1.03,MX,1706980912


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City','Country', 'Lat', 'Lng', 'Humidity']].copy() 
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
4,cabo san lucas,MX,22.8909,-109.9124,55,
5,maracas,BR,-13.4411,-40.4308,34,
9,fada,NG,9.5667,8.3000,14,
18,mbaiki,CF,3.8678,17.9892,27,
31,santa elena de uairen,VE,4.6167,-61.1333,32,
...,...,...,...,...,...,...
507,tanout,NE,14.9709,8.8879,11,
525,mul,IN,20.0667,79.6667,32,
546,lamarque,AR,-39.4230,-65.7021,15,
570,guerrero,MX,17.6667,-100.0000,37,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [12]:
# Set parameters to search for a hotel
radius = 10000
category = "accommodation.hotel"
params = {"categories":category,"apiKey":geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
        
    # Convert the API response to JSON format
    name_hotel = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_hotel["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
cabo san lucas - nearest hotel: Comfort Rooms
maracas - nearest hotel: Pousada Vale Aprazível
fada - nearest hotel: No hotel found
mbaiki - nearest hotel: No hotel found
santa elena de uairen - nearest hotel: Posada Villa Fairmont
chonchi - nearest hotel: Hotel Huildín
homosassa springs - nearest hotel: Bella Oasis Hotel
constitucion - nearest hotel: Hostal Alameda
bee ridge - nearest hotel: Comfort Suites Sarasota-Siesta Key
lihue - nearest hotel: Kauai Palms
ouahigouya - nearest hotel: Hotel de l'Amité
alice springs - nearest hotel: Aurora Alice Springs
illapel - nearest hotel: Domingo Ortiz de Rosas
nouadhibou - nearest hotel: Hotel valencia
concepcion - nearest hotel: Hotel One (ex-Biobio)
cururupu - nearest hotel: No hotel found
niquelandia - nearest hotel: Serrano Hotel
san luis de la loma - nearest hotel: No hotel found
villanueva - nearest hotel: Hotel Leiva
porto novo - nearest hotel: Hôtel 6500 CFA
springs - nearest hotel: Manhattan Hotel & Action Bar
br

,City,Country,Lat,Lng,Humidity,Hotel Name
4,cabo san lucas,MX,22.8909,-109.9124,55,Comfort Rooms
5,maracas,BR,-13.4411,-40.4308,34,Pousada Vale Aprazível
9,fada,NG,9.5667,8.3000,14,No hotel found
18,mbaiki,CF,3.8678,17.9892,27,No hotel found
31,santa elena de uairen,VE,4.6167,-61.1333,32,Posada Villa Fairmont
...,...,...,...,...,...,...
507,tanout,NE,14.9709,8.8879,11,No hotel found
525,mul,IN,20.0667,79.6667,32,No hotel found
546,lamarque,AR,-39.4230,-65.7021,15,No hotel found
570,guerrero,MX,17.6667,-100.0000,37,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
# %%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    x = 'Lng',
    y = 'Lat',
    size = 'Humidity',
    xlabel = 'Longitude',
    ylabel = 'Latitude',
    hover_cols = ['Hotel Name'],
    geo=True,
    tiles='OSM',
    color = 'City',
    scale = 0.5,
    frame_width = 700,
    frame_height = 500)

# Display the map
hotel_map


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name)